In [1]:
import anndata
import numpy as np
import scvelo as scv
import scanpy as sc
import sys
import torch
import os.path
import deepvelo as dv
import pickle as pickle
import matplotlib.pyplot as plt
import pandas as pd
import unitvelo as utv
from os.path import exists
method = 'DeepVelo'

(Running UniTVelo 0.2.5)
2023-05-16 14:55:44


In [2]:
datasets = ['Pancreas_with_cc', 'DentateGyrus' , 'MouseBoneMarrow', 'MouseErythroid', 'HumanBoneMarrow', 'HumanDevelopingBrain']
data_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_datasets/'
save_dir = '/nfs/team283/aa16/data/fate_benchmarking/benchmarking_results/'

In [ ]:
for dataset in datasets:
    print(dataset)
    adata = sc.read_h5ad(data_dir + dataset + '/' + dataset + '_anndata.h5ad')
    scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=3000)
    scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    trainer = dv.train(adata, dv.Constants.default_configs)
    scv.pp.neighbors(adata)
    scv.tl.velocity_graph(adata, vkey = 'velocity')
    scv.tl.velocity_embedding(adata, vkey = 'velocity')
    fix, ax = plt.subplots(1, 1, figsize = (8, 6))
    scv.pl.velocity_embedding_stream(adata, basis='umap', save = False, vkey='velocity',
                                     show = False, ax = ax)
    plt.savefig(save_dir + 'UMAPs/' + dataset + '_UMAP_DeepVelo.svg')
    # Calculate performance metrics:
    file = open(data_dir + dataset + '/' + dataset + '_groundTruth.pickle' ,'rb')
    ground_truth = pickle.load(file)
    metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
    if exists(save_dir + dataset + '_CBDC_scores.csv'):
        tab = pd.read_csv(save_dir + dataset + '_CBDC_scores.csv', index_col = 0)
    else:
        tab = pd.DataFrame(columns = list(metrics['Cross-Boundary Direction Correctness (A->B)'].keys()) + ['Mean'],
                 index = [method])
    cb_score = [np.mean(metrics['Cross-Boundary Direction Correctness (A->B)'][x])
                for x in metrics['Cross-Boundary Direction Correctness (A->B)'].keys()]
    tab.loc[method,:] = cb_score + [np.mean(cb_score)]
    tab.to_csv(save_dir + dataset + '_CBDC_scores.csv')
    metrics = utv.evaluate(adata, ground_truth, 'clusters', 'velocity')
    if exists(save_dir + dataset + '_ICC_scores.csv'):
        tab = pd.read_csv(save_dir + dataset + '_ICC_scores.csv', index_col = 0)
    else:
        tab = pd.DataFrame(columns = list(np.unique(np.concatenate(ground_truth))) + ['Mean'],
                 index = [method])
    icc_score = [np.mean(metrics['In-cluster Coherence'][x]) for x in np.unique(np.concatenate(ground_truth))]
    tab.loc[method,:] = icc_score + [np.mean(icc_score)]
    tab.to_csv(save_dir + dataset + '_ICC_scores.csv')
    fix, ax = plt.subplots(1, 1, figsize = (8, 6))
    scv.pl.velocity_embedding_stream(adata, basis='umap', save = False, vkey='velocity',
                                     show = False, ax = ax)
    plt.savefig(save_dir + 'UMAPs/' + dataset + '_UMAP_' + method + '.svg')

Pancreas_with_cc
Filtered out 20801 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 3000 highly variable genes.
Logarithmized X.
computing neighbors
    finished (0:00:11) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Config Warning: Set to use GPU, but GPU version of DGL is not installed. Reset to use CPU instead.
building graph


INFO:train:Beginning training of DeepVelo_Base ...


velo data shape: torch.Size([3696, 3000])


/nfs/team283/aa16/software/miniconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/nfs/team283/aa16/software/miniconda3/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
INFO:trainer:    epoch          : 1
INFO:trainer:    time:          : 10.626794576644897
INFO:trainer:    loss           : 98578.53125
INFO:trainer:    mse            : 2.556227922439575
INFO:trainer:    epoch          : 2
INFO:trainer:    time:          : 10.040715217590332
INFO:trainer:    loss           : 18010.15625
INFO:trainer:    mse            : 1.5266255140304565
INFO:trainer:    epoch          : 3
INFO:trainer:    time:          : 9.94972562789917
INFO:trainer:    loss           : 9833.1630859375
IN

INFO:trainer:    time:          : 10.01677393913269
INFO:trainer:    loss           : 2230.870361328125
INFO:trainer:    mse            : 0.5351672768592834
INFO:trainer:    epoch          : 42
INFO:trainer:    time:          : 10.1201012134552
INFO:trainer:    loss           : 2224.52734375
INFO:trainer:    mse            : 0.491735577583313
INFO:trainer:    epoch          : 43
INFO:trainer:    time:          : 10.051969051361084
INFO:trainer:    loss           : 2219.3623046875
INFO:trainer:    mse            : 0.48965513706207275
INFO:trainer:    epoch          : 44
INFO:trainer:    time:          : 10.045467376708984
INFO:trainer:    loss           : 2212.611572265625
INFO:trainer:    mse            : 0.4926551580429077
INFO:trainer:    epoch          : 45
INFO:trainer:    time:          : 10.014889001846313
INFO:trainer:    loss           : 2202.308349609375
INFO:trainer:    mse            : 0.49902263283729553
INFO:trainer:    epoch          : 46
INFO:trainer:    time:          :

...

    finished (0:00:05) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('dividing', 'progenitors'): -0.42908860070870725, ('progenitors', 'activating'): -0.5703333831148938}
Total Mean: -0.4997109919118005
# In-cluster Coherence
{'B cell lineage': 0.8856354, 'activating': 0.8885332, 'dividing': 0.79850686, 'macrophages': 0.90042406, 'progenitors': 0.83670336}
Total Mean: 0.8619605302810669
# Cross-Boundary Direction Correctness (A->B)
{('dividing', 'progenitors'): -0.42908860070870725, ('progenitors', 'activating'): -0.5703333831148938}
Total Mean: -0.4997109919118005
# In-cluster Coherence
{'B cell lineage': 0.8856354, 'activating': 0.8885332, 'dividing': 0.79850686, 'macrophages': 0.90042406, 'progenitors': 0.83670336}
Total Mean: 0.8619605302810669
MouseErythroid
Filtered out 47456

INFO:train:Beginning training of DeepVelo_Base ...


velo data shape: torch.Size([9815, 3000])


INFO:trainer:    epoch          : 1
INFO:trainer:    time:          : 26.259074449539185
INFO:trainer:    loss           : 163653.484375
INFO:trainer:    mse            : 1185.632080078125
INFO:trainer:    epoch          : 2
INFO:trainer:    time:          : 26.435172080993652
INFO:trainer:    loss           : 40205.65625
INFO:trainer:    mse            : 410.6335754394531
INFO:trainer:    epoch          : 3
INFO:trainer:    time:          : 26.729791164398193
INFO:trainer:    loss           : 41017.203125
INFO:trainer:    mse            : 590.7056884765625
INFO:trainer:    epoch          : 4
INFO:trainer:    time:          : 26.711156368255615
INFO:trainer:    loss           : 31551.3046875
INFO:trainer:    mse            : 468.87152099609375
INFO:trainer:    epoch          : 5
INFO:trainer:    time:          : 26.6386399269104
INFO:trainer:    loss           : 16632.234375
INFO:trainer:    mse            : 214.6481170654297
INFO:trainer:    epoch          : 6
INFO:trainer:    time:  

INFO:trainer:    loss           : 2396.504150390625
INFO:trainer:    mse            : 11.932602882385254
INFO:trainer:    epoch          : 45
INFO:trainer:    time:          : 26.627253770828247
INFO:trainer:    loss           : 2332.75390625
INFO:trainer:    mse            : 11.012228965759277
INFO:trainer:    epoch          : 46
INFO:trainer:    time:          : 26.66788601875305
INFO:trainer:    loss           : 2396.9052734375
INFO:trainer:    mse            : 12.52389907836914


In [ ]:
    fix, ax = plt.subplots(1, 1, figsize = (8, 6))
    scv.pl.velocity_embedding_stream(adata, basis='umap', save = False, vkey='velocity',
                                     show = False, ax = ax)
    plt.savefig(save_dir + 'UMAPs/' + dataset + '_UMAP_' + method + '.svg')